In [ ]:
# ===============================
# 🧹 SPAM DETECTION — DATA CLEANING
# ===============================

# 📦 Imports
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# ===============================
# 📂 1. Load Datasets
# ===============================
df1 = pd.read_csv("datasets/emails.csv")
df2 = pd.read_csv("datasets/spam.csv")

print("✅ Datasets loaded:")
print("emails.csv →", df1.shape)
print("spam.csv →", df2.shape)

# ===============================
# 🧾 2. Basic Cleaning
# ===============================
df2 = df2.rename(columns={"Category": "spam", "Message": "text"})
df2["spam"] = df2["spam"].map({"spam": 1, "ham": 0})

df = pd.concat([df1, df2], ignore_index=True)
df.drop_duplicates(subset="text", inplace=True)
df.dropna(subset=["text"], inplace=True)
print("✅ Combined dataset shape:", df.shape)

# ===============================
# 🧽 3. Advanced Cleaning Function
# ===============================
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text).lower()
    
    # --- Remove unwanted prefixes ---
    text = re.sub(r'^\s*(re|fwd|fw|subject)\s*:\s*', '', text, flags=re.IGNORECASE)
    
    # --- Remove URLs and HTML ---
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'<.*?>', '', text)
    
    # --- Remove digits and punctuation ---
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # --- Remove extra spaces ---
    text = re.sub(r'\s+', ' ', text).strip()
    
    # --- Remove stopwords and lemmatize ---
    tokens = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    text = " ".join(tokens)
    
    return text

# Apply cleaning
df["clean_text"] = df["text"].apply(clean_text)

# ===============================
# 🧠 4. Verify Cleaning
# ===============================
print("\n🔍 Before vs After Cleaning:")
for i in range(3):
    print(f"\n📩 Original: {df['text'].iloc[i][:100]}")
    print(f"🧼 Cleaned : {df['clean_text'].iloc[i][:100]}")

# ===============================
# 💾 5. Save Cleaned Dataset
# ===============================
df.to_csv("cleaned_spam_dataset.csv", index=False)
print("\n💾 Saved cleaned dataset as 'cleaned_spam_dataset.csv'")
print("✅ Cleaning complete!")



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhyu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abhyu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✅ Datasets loaded:
emails.csv → (5728, 2)
spam.csv → (5572, 2)
✅ Combined dataset shape: (10852, 2)

🔍 Before vs After Cleaning:

📩 Original: Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : 
🧼 Cleaned : naturally irresistible corporate identity lt really hard recollect company market full suqgestions i

📩 Original: Subject: the stock trading gunslinger  fanny is merrill but muzo not colza attainder and penultimate
🧼 Cleaned : stock trading gunslinger fanny merrill muzo colza attainder penultimate like esmark perspicuous ramb

📩 Original: Subject: unbelievable new homes made easy  im wanting to show you this  homeowner  you have been pre
🧼 Cleaned : unbelievable new home made easy im wanting show homeowner pre approved home loan fixed rate offer ex

💾 Saved cleaned dataset as 'cleaned_spam_dataset.csv'
✅ Cleaning complete!
